In [1]:
import requests
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
import numpy as np
import pandas as pd
import folium 
from folium.plugins import HeatMap

In [2]:
client_id = 103696
client_secret = "95fe1291bbff71246ca7a3c87fa7327e874cadf2"
scope = 'activity:read_all'
url = "https://www.strava.com/oauth/authorize"
token_url = "https://www.strava.com/oauth/token"

In [3]:
strava = OAuth2Session(client_id=client_id, redirect_uri='http://127.0.0.1', scope=scope)
authorization_url, state = strava.authorization_url(url)
print(authorization_url)

https://www.strava.com/oauth/authorize?response_type=code&client_id=103696&redirect_uri=http%3A%2F%2F127.0.0.1&scope=activity%3Aread_all&state=rQptnf8DKSNf7hNX2anaWu5GKWqp3Q


In [4]:
redirect_response = "http://127.0.0.1/?state=rQptnf8DKSNf7hNX2anaWu5GKWqp3Q&code=bf01f81b9b50f1a545d8decf9a3ec0621f861420&scope=read,activity:read_all"

In [5]:
code = str(redirect_response.split("&code=")[1].split("&scope=")[0])
print(redirect_response)
print(code)

http://127.0.0.1/?state=rQptnf8DKSNf7hNX2anaWu5GKWqp3Q&code=bf01f81b9b50f1a545d8decf9a3ec0621f861420&scope=read,activity:read_all
bf01f81b9b50f1a545d8decf9a3ec0621f861420


In [6]:
data = {
    "client_id": client_id,
    "code": code,
    "client_secret": client_secret,
}

response = requests.post(token_url, data=data)
print(response)
response.content

<Response [200]>


b'{"token_type":"Bearer","expires_at":1678480445,"expires_in":21600,"refresh_token":"62acbcb2e5da7f90876baeee3cbf5128fdc16e21","access_token":"7ac5dbca09cc0adff334d5bee41670326528d372","athlete":{"id":73875527,"username":null,"resource_state":2,"firstname":"miles","lastname":"krebs-moberg","bio":null,"city":null,"state":null,"country":null,"sex":"M","premium":false,"summit":false,"created_at":"2020-12-15T14:54:22Z","updated_at":"2023-03-06T22:51:00Z","badge_type_id":0,"weight":0.0,"profile_medium":"https://dgalywyr863hv.cloudfront.net/pictures/athletes/73875527/25115074/1/medium.jpg","profile":"https://dgalywyr863hv.cloudfront.net/pictures/athletes/73875527/25115074/1/large.jpg","friend":null,"follower":null}}'

In [7]:
if response.status_code in range(2,299):
    response = response.json()
    token = {
        "access_token": response["access_token"],
        "refresh_token": response["refresh_token"],
        "token_type": response["token_type"],
        "expires_in": response["expires_in"],
        "expires_at": response['expires_at']
    }
    

In [8]:
token

{'access_token': '7ac5dbca09cc0adff334d5bee41670326528d372',
 'refresh_token': '62acbcb2e5da7f90876baeee3cbf5128fdc16e21',
 'token_type': 'Bearer',
 'expires_in': 21600,
 'expires_at': 1678480445}

In [10]:
def refresh_token():
    data = {
    "client_id": client_id,
    "refresh_token": token['refresh_token'],
    "client_secret": client_secret,
    "grant_type": "refresh_token"
    }

    response = requests.post(token_url, data=data)
    print(response)
    return response.content

In [3]:
print(refresh_token())

NameError: name 'refresh_token' is not defined

In [11]:
url = "https://www.strava.com/api/v3/athlete/activities?per_page=30"
headers = {
    "accepts": "application/json",
    "Authorization": f"Bearer {token['access_token']}"
}

activity_r = requests.get(url, headers=headers)
activity_r.status_code

200

In [12]:
activity_r.json()

[{'resource_state': 2,
  'athlete': {'id': 73875527, 'resource_state': 1},
  'name': 'I’m fucking crazy, but I’m free ✨',
  'distance': 176100.0,
  'moving_time': 26019,
  'elapsed_time': 32694,
  'total_elevation_gain': 1995.0,
  'type': 'Ride',
  'sport_type': 'Ride',
  'workout_type': 10,
  'id': 7700824552,
  'start_date': '2022-08-25T15:01:40Z',
  'start_date_local': '2022-08-25T08:01:40Z',
  'timezone': '(GMT-08:00) America/Los_Angeles',
  'utc_offset': -25200.0,
  'location_city': None,
  'location_state': None,
  'location_country': None,
  'achievement_count': 272,
  'kudos_count': 2,
  'comment_count': 0,
  'athlete_count': 1,
  'photo_count': 0,
  'map': {'id': 'a7700824552',
   'summary_polyline': '}ileFj~fjVmAnx@q[|AtDpWySrT}e@vFXlLiUtCl@fPcNxNkKp~@hKnd@gEjVbF`Y}IpYcrA`EyDkbAgx@pS_JcEuP~GeUiNmRfCmAuGeLpMmfCzN_XfXkYkGwH_^}_@rBoEzO{ZgKq^~F_]h|@cBdZmHlPwr@fm@wbAbyAhRvW_HpUbKnMiErv@}]v@w[mSm@lNxI~IkHfPdNbFsCxJnCdh@dTpPb[kAb[za@UbNd[rGvWrLrS|l@{Vh\\tDtEpSmHgNp`@hFzMeBtJ{R`F{h@n

In [14]:
ids = {
    "arcada": 7633975856,
    "century": 7700824552,
    "first": 7538518690
}

In [26]:
def fetch_activity_stream(id):
    url = f"https://www.strava.com/api/v3/activities/{id}/streams?keys=latlng&key_by_type=true"
    headers = {
        "accepts": "application/json",
        "Authorization": f"Bearer {token['access_token']}"
    }

    latlong = requests.get(url, headers=headers)

    
    return latlong

In [68]:
coords = [tuple(x) for x in total_sorted[['lat', 'long']].to_numpy()]
coords[:10]

[(37.751465, -122.43175),
 (37.751467, -122.431686),
 (37.751474, -122.431618),
 (37.751479, -122.431809),
 (37.751487, -122.431537),
 (37.751493, -122.431474),
 (37.751498, -122.431415),
 (37.751505, -122.431345),
 (37.751512, -122.429335),
 (37.751513, -122.431858)]

In [125]:
first_df = None
with open ('first.csv', 'r') as first_data:
    first_df = pd.read_csv(first_data, header=0)
first_df.head()

,lat,long
0,37.758350,-122.421394
1,37.758352,-122.421392
2,37.758359,-122.421388
3,37.758364,-122.421380
4,37.758368,-122.421367


In [135]:
first_df = first_df.apply(np.floor)
first_df.head()

,lat,long
0,37.0,-123.0
1,37.0,-123.0
2,37.0,-123.0
3,37.0,-123.0
4,37.0,-123.0


In [133]:
m = folium.Map(location=[37.767596, -122.443876],
                tiles = "OpenStreetMap",
                zoom_start = 10)
pop = folium.LatLngPopup()
pop.add_to(m)


In [134]:
folium.PolyLine(first_df, weight=3, color="#5a9e6c", tooltip="first").add_to(m)


m